In [1]:
from collections import defaultdict
import math
import time
import random
import os, sys

import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np


from torch.utils.data import Dataset, DataLoader,TensorDataset
torch.manual_seed(1)    # reproducible

In [2]:
K=3 #number of negative samples
N = 2 # The length of the n-gram
EMB_SIZE = 64 # The size of the embedding
BATCH_SIZE=512

embeddings_location = "embeddings.txt" #the file to write the word embeddings to
labels_location = "labels.txt" #the file to write the labels to

USE_CUDA = torch.cuda.is_available()

# Functions to read in the corpus
# NOTE: We are using data from the Penn Treebank, which is already converted
#       into an easy-to-use format with "<unk>" symbols. If we were using other
#       data we would have to do pre-processing and consider how to choose
#       unknown words, etc.
w2i = defaultdict(lambda: len(w2i))
S = w2i["<s>"]
UNK = w2i["<unk>"]

#word counts for negative sampling
word_counts = defaultdict(int)

def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            for word in line:
                word_counts[w2i[word]] += 1
            yield [w2i[x] for x in line.strip().split(" ")]

# Read in the data
train = list(read_dataset("../data/ptb/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/ptb/valid.txt"))
i2w = {v: k for k, v in w2i.items()}
nwords = len(w2i)


# take the word counts to the 3/4, normalize
counts =  np.array([list(x) for x in word_counts.items()])[:,1]**.75
normalizing_constant = sum(counts)
word_probabilities = np.zeros(nwords)
for word_id in word_counts:
    word_probabilities[word_id] = word_counts[word_id]**.75/normalizing_constant


with open(labels_location, 'w') as labels_file:
    for i in range(nwords):
        labels_file.write(i2w[i] + '\n')

In [3]:

class SKIP_NS(nn.Module):
    def __init__(self, nwords, emb_size):
        super(SKIP_NS, self).__init__()
        self.embedding = nn.Embedding(nwords, emb_size)

    def forward(self, words):
        #print(words)
        emb=self.embedding(words)    # 3D Tensor of size [batch_size x 2x emb_size]
        #print(emb1*emb2)
        return (emb[:,0,:]*emb[:,1,:]).sum(1)

In [4]:
# Initialize the model and the optimizer
model = SKIP_NS(nwords=nwords, emb_size=EMB_SIZE)
if USE_CUDA:
    model = model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [5]:
# convert a (nested) list of int into a pytorch tensor
def convert_to_tensor(words):
    var = torch.LongTensor(words)
#     if USE_CUDA:
#         var = var.cuda()
    return var

# convert a (nested) list of int into a pytorch Variable
def convert_to_variable(tensor):
    var = Variable(tensor)
    if USE_CUDA:
        var = var.cuda()

    return var

def convert_to_loader(train):
    
    all_histories = []
    all_targets = []
    for sent in train:
        if len(sent)==0:
            continue
        all_neg_words = np.random.choice(nwords, size=2*N*K*len(sent), replace=True, p=word_probabilities).tolist()
        # The initial history is equal to end of sentence symbols
        padded_sent = [S] * N + sent + [S] * N
        # Step through the sentence, including the end of sentence token

        for i in range(0,len(sent)):
            neg_words = all_neg_words[i*K*2*N:(i+1)*K*2*N]
            pos_words = ([sent[x] if x >= 0 else S for x in range(i-N,i)] +
                     [sent[x] if x < len(sent) else S for x in range(i+1,i+N+1)])
            for pos_word in pos_words:
                all_histories.append([sent[i],pos_word])
                all_targets.append(1)
            for neg_word in neg_words:
                all_histories.append([sent[i],neg_word])         
                all_targets.append(0)



    train_dataset = TensorDataset(data_tensor=convert_to_tensor(all_histories), 
                                       target_tensor=convert_to_tensor(all_targets))
    train_loader = DataLoader(
        dataset=train_dataset,      # torch TensorDataset format
        batch_size=BATCH_SIZE,      # mini batch size
        shuffle=True,               
        num_workers=0
    )
    return train_loader

train_loader=convert_to_loader(train)
dev_loader=convert_to_loader(dev)


In [6]:

last_dev = 1e20
best_dev = 1e20

for ITER in range(5):
    # Perform training
    model.train()
    train_words, train_loss = 0, 0.0
    start = time.time()
    for step, (batch_x, batch_y) in enumerate(train_loader):
        
        batch_x=convert_to_variable(batch_x)
        batch_y=convert_to_variable(batch_y.float())
        #print(sent)
        predictions=model(batch_x)
        my_loss = nn.functional.binary_cross_entropy_with_logits(predictions, batch_y,size_average=False)
        train_loss += my_loss.data[0]
        train_words += len(batch_y)
        optimizer.zero_grad()
        my_loss.backward()
        optimizer.step()
    print("iter %r: train loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (ITER, train_loss/train_words, math.exp(train_loss/train_words), train_words/(time.time()-start)))

    # Evaluate on dev set
    # set the model to evaluation mode
    model.eval()
    dev_words, dev_loss = 0, 0.0
    start = time.time()
    for step,(batch_x, batch_y) in enumerate(dev_loader):
        batch_x=convert_to_variable(batch_x)
        batch_y=convert_to_variable(batch_y.float())
        predictions=model(batch_x)
        my_loss = nn.functional.binary_cross_entropy_with_logits(predictions, batch_y,size_average=False)
        dev_loss += my_loss.data[0]
        dev_words += len(batch_y)

    # Keep track of the development accuracy and reduce the learning rate if it got worse
    if last_dev < dev_loss:
        for g in optimizer.param_groups:
            g['lr']/=2
        print('lr decay')
    last_dev = dev_loss

    # Keep track of the best development accuracy, and save the model only if it's the best one
    if best_dev > dev_loss:
        print("saving embedding files")
        with open(embeddings_location, 'w') as embeddings_file:
            W_w_np = model.embedding.weight.cpu().data.numpy()
            for i in range(nwords):
                ith_embedding = '\t'.join(map(str, W_w_np[i]))
                embeddings_file.write(ith_embedding + '\n')
        best_dev = dev_loss
        
        # Save the model
    print("iter %r: dev loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (ITER, dev_loss/dev_words, math.exp(dev_loss/dev_words), dev_words/(time.time()-start)))



iter 0: train loss/word=0.1092, ppl=1.1153 (word/sec=122282.94)
saving embedding files
iter 0: dev loss/word=0.0820, ppl=1.0854 (word/sec=126529.83)
iter 1: train loss/word=0.0685, ppl=1.0709 (word/sec=105992.17)
saving embedding files
iter 1: dev loss/word=0.0746, ppl=1.0774 (word/sec=141617.11)
iter 2: train loss/word=0.0657, ppl=1.0679 (word/sec=101518.63)
saving embedding files
iter 2: dev loss/word=0.0738, ppl=1.0766 (word/sec=150767.95)
iter 3: train loss/word=0.0642, ppl=1.0663 (word/sec=103025.02)
saving embedding files
iter 3: dev loss/word=0.0711, ppl=1.0737 (word/sec=106952.10)
iter 4: train loss/word=0.0635, ppl=1.0656 (word/sec=98716.45)
lr decay
iter 4: dev loss/word=0.0774, ppl=1.0804 (word/sec=113988.57)
